In [10]:
import numpy as np; π = np.pi
import sympy as sp

def convert_to_numpy(f):
    f = str(f)
    subs = [('u(t)', 'u'), 
            ('Derivative(θ1(t), (t, 2))', 'ddθ1'), 
            ('Derivative(θ2(t), (t, 2))', 'ddθ2'), 
            ('Derivative(θ2(t), t)', 'dθ2'), 
            ('Derivative(θ1(t), t)', 'dθ1'), 
            ('θ2(t)', 'θ2'), 
            ('θ1(t)', 'θ1'), 
            # ('**', '^'),
    ]
    for s in subs: f = f.replace(*s)
    return f

In [11]:
# SINGLE PENDULUM #################################################################################################
###################################################################################################################
l1 = 1 # length of pendulum
g = 9.81 # gravity
μ1 = 0.7 # damping coefficient
m1 = 1 # mass of pendulum

np_ddθ1 = f'({g*l1*m1}*sin(θ1) - {μ1}*dθ1 + u)/{l1**2*m1}'
print(f'ddθ1 = {np_ddθ1}')

#create a .py file called single_pendulum_eqns.py with the equations of motion, and the parameters
with open('eqns_single_pendulum.py', 'w') as file:
    file.write(f'from numpy import sin\n\n')
    file.write(f'def f_ddθ1(θ1, dθ1, u):\n')
    file.write(f'    return {np_ddθ1}\n\n')
    file.write(f'l1 = {l1}\n')
    file.write(f'g = {g}\n')
    file.write(f'μ1 = {μ1}\n')
    file.write(f'm1 = {m1}\n')

ddθ1 = (9.81*sin(θ1) - 0.7*dθ1 + u)/1


In [12]:
# DOUBLE PENDULUM #################################################################################################
###################################################################################################################
l1 = 1  # First arm
l2 = 1  # Second arm
g = 9.81  # gravity
μ1 = .7 #.7  # friction coefficient first joint
μ2 = .7 #.7  # friction coefficient second joint
m1 = 1  # mass of the first pendulum
m2 = 1  # mass of the second pendulum

# use lagrangian mechanics to derive the equations of motion
# define the symbolic variables
t = sp.symbols('t')
θ1, θ2, u = sp.symbols('θ1 θ2 u', cls=sp.Function)
#define as functions of time
θ1, θ2 = θ1(t), θ2(t) # angles of the joints
u = u(t) # control input
dθ1, dθ2 = θ1.diff(t), θ2.diff(t) # angular velocities of the joints
ddθ1, ddθ2 = dθ1.diff(t), dθ2.diff(t) # angular accelerations of the joints

#define position of all the masses
x1, y1 = l1*sp.sin(θ1), l1*sp.cos(θ1) # position of the first pendulum
x2, y2 = x1 + l2*sp.sin(θ2), y1 + l2*sp.cos(θ2) # position of the second pendulum
dx1, dy1 = x1.diff(t), y1.diff(t) # velocity of the first pendulum
dx2, dy2 = x2.diff(t), y2.diff(t) # velocity of the second pendulum

# define the kinetic energy of the system
T1 = 1/2*m1*(dx1**2 + dy1**2) # kinetic energy of the first pendulum
T2 = 1/2*m2*(dx2**2 + dy2**2) # kinetic energy of the second pendulum
T = T1 + T2 # total kinetic energy

# define the potential energy of the system
V1 = m1*g*y1 # potential energy of the first pendulum
V2 = m2*g*y2 # potential energy of the second pendulum
V = V1 + V2 # total potential energy

# lagrangian
L = T - V

# get the lagrange equations
LEQθ1 = L.diff(θ1) - (L.diff(dθ1)).diff(t) - μ1*dθ1 + u # lagrange equation for the first joint
LEQθ2 = L.diff(θ2) - (L.diff(dθ2)).diff(t) - μ2*dθ2 # lagrange equation for the second join

# lambdify the equations of motion
sol = sp.solve([LEQθ1, LEQθ2], [ddθ1, ddθ2], simplify=False)
ddθ1 = sol[ddθ1].simplify()
ddθ2 = sol[ddθ2].simplify()

# convert equations to numpy
np_ddθ1 = convert_to_numpy(ddθ1)
np_ddθ2 = convert_to_numpy(ddθ2)
print(f'ddθ1 = {np_ddθ1}')
print(f'ddθ2 = {np_ddθ2}')

#create a .py file called double_pendulum_eqns.py with the equations of motion, and the parameters
with open('eqns_double_pendulum.py', 'w') as file:
    file.write(f'from numpy import sin, cos, tan, exp, log, sqrt, pi\n')
    file.write(f'def f_ddθ1(θ1, θ2, dθ1, dθ2, u):\n')
    file.write(f'    return {np_ddθ1}\n\n')
    file.write(f'def f_ddθ2(θ1, θ2, dθ1, dθ2, u):\n')
    file.write(f'    return {np_ddθ2}\n\n')
    file.write(f'l1 = {l1}\n')
    file.write(f'l2 = {l2}\n')
    file.write(f'g = {g}\n')
    file.write(f'μ1 = {μ1}\n')
    file.write(f'μ2 = {μ2}\n')
    file.write(f'm1 = {m1}\n')
    file.write(f'm2 = {m2}\n')

ddθ1 = (-0.5*u - 2.4525*sin(θ1 - 2*θ2) + 0.5*sin(θ1 - θ2)*dθ2**2 + 0.25*sin(2*θ1 - 2*θ2)*dθ1**2 - 7.3575*sin(θ1) - 0.35*cos(θ1 - θ2)*dθ2 + 0.35*dθ1)/(0.25*cos(2*θ1 - 2*θ2) - 0.75)
ddθ2 = (0.5*u*cos(θ1 - θ2) - 1.0*sin(θ1 - θ2)*dθ1**2 - 0.25*sin(2*θ1 - 2*θ2)*dθ2**2 + 4.905*sin(2*θ1 - θ2) - 4.905*sin(θ2) - 0.35*cos(θ1 - θ2)*dθ1 + 0.7*dθ2)/(0.25*cos(2*θ1 - 2*θ2) - 0.75)
